## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..
import os, sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))

/Users/Tony/Other Docs/distilling-and-forgetting-in-large-pre-trained-models


In [3]:
from IPython.display import Audio, display
from tqdm.auto import tqdm

import numpy as np
import torch
import whisper
import datasets

import matplotlib.pyplot as plt
import seaborn as sns

from evaluation.eval_dataset_name_to_dataset_group import EVAL_DATASET_NAME_TO_DATASET_GROUP

device = torch.device('mps')
sns.set_theme(context="paper", style="ticks")

/opt/homebrew/Caskroom/miniforge/base/envs/mlmi-dissertation/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


## Load dataset

In [6]:
dataset_name = "ami"

ds_group = EVAL_DATASET_NAME_TO_DATASET_GROUP[dataset_name]()

if dataset_name == "librispeech_dummy":
    ds = ds_group.str2dataset["librispeech_dummy"]
    ds = ds.map(lambda x: {"text": x.lower()}, input_columns=["text"])
elif dataset_name in ["ami", "ami_10h"]:
    ds = ds_group.str2dataset["ami"]
    ds = ds.map(lambda x: {"text": x.lower()}, input_columns=["text"])
else:
    raise ValueError()

Found cached dataset ami (/Users/Tony/.cache/huggingface/datasets/edinburghcstr___ami/ihm/0.0.0/0d128d0aa8145d0f16f3d5b4da86c5d5759dbe9e8f947fda04b25edb56442bd5)
Loading cached processed dataset at /Users/Tony/.cache/huggingface/datasets/edinburghcstr___ami/ihm/0.0.0/0d128d0aa8145d0f16f3d5b4da86c5d5759dbe9e8f947fda04b25edb56442bd5/cache-818d7a4553237f1c.arrow


In [7]:
# Fill `list_idx_hallucination` according to previous analysis:
list_idx_hallucination = [1941, 1954, 3311, 3691, 3808, 3917, 4464, 10450, 11468]

ds_hallucinations = ds.select(indices=list_idx_hallucination)

In [10]:
list_audio = [sample["audio"] for sample in ds_hallucinations]
references = [sample["text"].lower() for sample in ds_hallucinations]

In [12]:
for idx, audio, ref in zip(list_idx_hallucination, list_audio, references):
    print(f"Idx: {idx}")
    print(f"Reference: {ref}")
    display(Audio(data=audio["array"], rate=audio["sampling_rate"]))
    print()

Idx: 1941
Reference: you know splits that thing into like two or three or whatever that is



Idx: 1954
Reference: oh right



Idx: 3311
Reference: yeah



Idx: 3691
Reference: right okay



Idx: 3808
Reference: yeah



Idx: 3917
Reference: it's just much bigger than all of our home directories combined isn't it



Idx: 4464
Reference: right



Idx: 10450
Reference: forty



Idx: 11468
Reference: so then we get
